In [ ]:
import wave
import numpy as np
import pylab as plt
from scipy.signal import firwin
from scipy.signal.signaltools import get_window
from scipy.signal import iirnotch, lfilter
from numpy.fft import rfft
%matplotlib inline
np.set_printoptions(precision=4)

In [ ]:
def sample_to_int(data):
        d = int.from_bytes(data, byteorder='little', signed=True)
        return d

def int_to_sample(data):
        d = int(data)
        return d.to_bytes(3, byteorder='little', signed=True)

In [ ]:
def calc_thd(data):
    f = rfft(data)
    i = np.argmax(abs(f))
    fs = i/len(data)
    freq = SR*fs
    b, a = iirnotch(fs*2, 1)
    filtered_data = lfilter(b, a, data)
    filtered_data = filtered_data[1000:]
    for i in range(len(filtered_data)):
        f_out.writeframes(int_to_sample(filtered_data[i]))
    rms_in = np.sqrt(np.mean(np.square(data)))
    rms = np.sqrt(np.mean(np.square(filtered_data)))
    thd = 20*np.log10(rms/rms_in)
    print("Frequency: %f THD+N: %.1f dB" % (freq, thd))

In [ ]:
SR = 48000
SEC = 1
N = int(SR*SEC)
fname = "wave_out.wav"
f = wave.open(fname, 'rb')

f_out = wave.open("thd.wav", "wb")
f_out.setframerate(SR)
f_out.setsampwidth(3)
f_out.setnchannels(1)

data = np.zeros(N)
work = True
while(work):
    for i in range(N):
        d = f.readframes(1)
        if(len(d)==0):
            work = False
            break
        data[i] = sample_to_int(d)
    calc_thd(data)

In [ ]:
f.close()
f_out.close()